In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7129,2024-06-09,Eua Nba,21:00,Boston Celtics,Dallas Mavericks,1.38,3.22,213.5,1.81,2.04,-7.5,2.02,7.00,xCWwLwZg,0.724638,0.310559,0.552486,0.490196,0.035197,0.6,0.6,NaN,NaN,141.252,15.070410,0.106692,248.778,60.495294,0.243170,154.468,259.232,163.59,331.08,0.0,0.0,0.0,0.0,0.565685,0.084485,0.780796,0.87,0.174,2.183908,0.812804,0.8,-0.012804,4.39,0.878,2.528474,0.495189,0.9,0.404811
7130,2024-06-09,Austrália Nbl1 Norte,02:00,Logan Thunder,Mackay Meteors,2.03,1.71,182.5,1.85,1.85,-4.5,2.02,1.98,CQxQNE26,0.492611,0.584795,0.540541,0.540541,0.077406,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,204.97,0.0,0.0,0.0,0.0,0.121002,0.000000,0.014142,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7131,2024-06-09,Eua Wnba,17:00,Dallas Wings F,Phoenix Mercury F,1.69,2.19,163.5,1.80,1.96,-3.5,2.00,2.72,KxTVpciU,0.591716,0.456621,0.555556,0.510204,0.048337,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.182244,0.060179,0.215727,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7132,2024-06-09,Eua Wnba,20:00,Minnesota Lynx F,Seattle Storm F,1.65,2.26,161.5,1.82,1.92,-3.5,1.95,2.68,WA9y6tNu,0.606061,0.442478,0.549451,0.520833,0.048538,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,221.34,175.10,0.0,0.0,0.0,0.0,0.220632,0.037813,0.222975,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7133,2024-06-09,Filipinas Copa Das Filipinas,07:15,San Miguel Beermen,Meralco Bolts,1.56,2.40,185.5,1.80,1.86,-4.5,1.87,3.90,xEepYdF7,0.641026,0.416667,0.555556,0.537634,0.057692,0.0,0.4,NaN,NaN,NaN,NaN,NaN,180.324,14.735504,0.081717,0.000,166.422,0.00,190.92,0.0,0.0,0.0,0.0,0.299985,0.023184,0.497548,0.00,0.000,inf,0.000000,0.0,0.000000,2.08,0.416,3.365385,0.000000,0.0,0.000000
7134,2024-06-09,França Lnb,13:30,Paris,Monaco,1.89,1.92,163.5,1.82,1.96,-1.5,2.01,2.04,ClRTwWNO,0.529101,0.520833,0.549451,0.510204,0.049934,0.6,0.4,NaN,NaN,167.820,64.085992,0.381873,119.284,28.954743,0.242738,121.152,123.934,95.70,148.63,0.0,0.0,1.0,0.0,0.011136,0.052378,0.010476,-0.05,-0.010,-89.000000,0.000000,0.0,0.000000,0.88,0.176,5.227273,0.000000,0.0,0.000000
7135,2024-06-09,Itália Série A2,13:00,Agrigento,Chiusi,1.91,1.81,151.5,1.88,1.79,5.5,1.40,1.42,EV8DqJ3B,0.523560,0.552486,0.531915,0.558659,0.076046,0.6,0.4,NaN,NaN,183.628,39.841726,0.216970,155.210,23.236820,0.149712,230.208,330.634,168.27,130.90,0.0,0.0,0.0,0.0,0.038016,0.034681,0.010030,-2.97,-0.594,-1.531987,0.000000,0.0,0.000000,4.75,0.950,0.852632,0.000000,0.0,0.000000
7136,2024-06-09,Venezuela Superliga,19:00,Llaneros,Spartans,2.02,1.72,160.5,1.85,1.85,1.5,1.87,1.59,SK5Fk1qp,0.495050,0.581395,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,182.16,160.60,0.0,0.0,0.0,0.0,0.113440,0.000000,0.114445,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7137,2024-06-09,Canadá Cebl,15:00,Brampton Honey Badgers,Scarborough Shooting Stars,1.92,1.80,178.5,1.85,1.85,-1.5,1.98,1.85,MwPhiqwR,0.520833,0.555556,0.540541,0.540541,0.076389,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.045620,0.000000,0.048002,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7138,2024-06-09,Canadá Cebl,17:00,Calgary Surge,Winnipeg Sea Bears,1.97,1.76,181.5,1.85,1.85,-1.5,1.98,1.81,SK1Ln3or,0.507614,0.568182,0.540541,0.540541,0.075796,0.0,0.0,NaN,NaN

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Boston Celtics,Dallas Mavericks,213.5,1.81,1.0,Over
1,02:00,Austrália Nbl1 Norte,Logan Thunder,Mackay Meteors,182.5,1.85,1.0,Over
2,17:00,Eua Wnba,Dallas Wings F,Phoenix Mercury F,163.5,1.80,1.0,Over
3,20:00,Eua Wnba,Minnesota Lynx F,Seattle Storm F,161.5,1.82,1.0,Over
4,07:15,Filipinas Copa Das Filipinas,San Miguel Beermen,Meralco Bolts,185.5,1.80,1.0,Over
5,13:30,França Lnb,Paris,Monaco,163.5,1.82,1.0,Over
